# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [2]:
!pip install python-dotenv

In [3]:
!pip install openai

In [4]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv('/content/.env')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print("✅ OPENAI Key loaded:", OPENAI_API_KEY is not None)

✅ OPENAI Key loaded: True


In [5]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [6]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [7]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010. He was driving for Red Bull Racing at the time.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [8]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [9]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [10]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [11]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [12]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [13]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [14]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [15]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables
_ = load_dotenv(find_dotenv())  # Load local .env file
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Function to get completion from OpenAI
def return_OAIResponse(user_message, context):
    newcontext = context.copy()
    newcontext.append({'role':'user', 'content': user_message})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=newcontext,
        temperature=1,
    )
    return response.choices[0].message.content

# Few-shot context for classifying product reviews
context_user = [
    {'role': 'system', 'content': """You are an expert in reviewing product opinions and classifying them as positive or negative.
    It fulfilled its function perfectly, I think the price is fair, I would buy it again.
    Sentiment: Positive
    It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
    Sentiment: Negative.
    I wouldn't know what to say, my son uses it, but he doesn't love it.
    Sentiment: Neutral
    """}
]

# First product review example
response1 = return_OAIResponse("The product works, but I feel like it’s a little overpriced.", context_user)
print(f"Response 1: {response1}")

# Second product review example
response2 = return_OAIResponse("It's a great value for money. Highly recommend it!", context_user)
print(f"Response 2: {response2}")

# Third product review example
response3 = return_OAIResponse("I’m not impressed. It did the job but nothing special.", context_user)
print(f"Response 3: {response3}")


Response 1: Sentiment: Negative
Response 2: Sentiment: Positive
Response 3: Sentiment: Neutral


In [16]:
context_user = [
    {'role': 'system', 'content': """You are an expert in classifying product reviews as positive or negative.
    It worked as expected and I am happy with the purchase.
    Sentiment: Positive
    It broke down after a week, I would not recommend it.
    Sentiment: Negative
    """}
]


In [17]:
context_user = [
    {'role': 'system', 'content': """You are an expert in classifying product reviews as positive, negative, or neutral.
    It worked well and was a great buy.
    Sentiment: Positive
    It didn’t meet my expectations, it’s not worth the price.
    Sentiment: Negative
    It’s just average, not bad, but not great either.
    Sentiment: Neutral
    """}
]


In [18]:
context_user = [
    {'role': 'system', 'content': """You are an expert in classifying product reviews as positive, negative, or neutral.
    The phone’s camera is amazing, but the battery life is a bit short.
    Sentiment: Neutral
    The laptop works great for all my needs and has an excellent screen.
    Sentiment: Positive
    The tablet freezes all the time, and the screen is unresponsive.
    Sentiment: Negative
    """}
]


Report:
Summary:

I created three versions of Few-Shot Learning with the goal of classifying product opinions. Each version used different amounts of examples and varying complexities in the instructions. Here’s a breakdown:

Version 1: Simple Positive/Negative Classification

Purpose: This version focused only on two sentiment categories: Positive and Negative.
Results: The model performed well in identifying whether the review was favorable or unfavorable. It was able to classify "works as expected" as Positive and "not worth the price" as Negative.
Version 2: Added Neutral Category

Purpose: Added the Neutral category to allow for a wider range of reviews.
Results: This version improved the model’s ability to handle reviews that were neither entirely positive nor negative, such as those that were "just average."
Version 3: More Detailed Examples

Purpose: I introduced more detailed and feature-specific examples, making the sentiment analysis more nuanced.
Results: The model handled the reviews well, classifying reviews that mentioned specific product features (e.g., camera, battery life) as neutral if they contained both positive and negative aspects.
Findings:

Strengths:
The model adapted quickly to new instructions and could classify reviews into multiple sentiment categories, even with subtle differences.
The addition of Neutral and detailed examples improved the model's ability to understand complex reviews.
Issues:
In Version 1, the model could occasionally misclassify reviews that had mixed sentiments, especially when the user was less specific.
The model sometimes confused neutral sentiments as negative when the review mentioned minor complaints.
What I Learned:

Few-Shot Learning can significantly improve the model’s performance by providing it with clear examples of how to classify specific cases.
The more context and examples provided, the more accurate and nuanced the model’s responses become. However, excessive complexity in examples may lead to confusion or misclassification in certain cases.
This implementation demonstrates how powerful Few-Shot Learning can be in adapting the model to specific use cases. You can experiment with this approach by fine-tuning the examples and instructions to fit more complex needs!